In [ ]:
!pip install d2l==1.0.0-beta0

Trong Bảng 1.5.1 , chúng tôi đã thảo luận về sự phát triển nhanh chóng của tính toán trong hai thập kỷ qua. Tóm lại, hiệu suất GPU đã tăng lên gấp 1000 lần sau mỗi thập kỷ kể từ năm 2000. Điều này mang lại những cơ hội tuyệt vời nhưng cũng cho thấy nhu cầu đáng kể để cung cấp hiệu suất như vậy.

Trong phần này, chúng ta bắt đầu thảo luận về cách khai thác hiệu suất tính toán này cho nghiên cứu của bạn. Đầu tiên bằng cách sử dụng các GPU đơn lẻ và sau đó là cách sử dụng nhiều GPU và nhiều máy chủ (với nhiều GPU).

Cụ thể, chúng tôi sẽ thảo luận về cách sử dụng một GPU NVIDIA duy nhất để tính toán. Trước tiên, hãy đảm bảo rằng bạn đã cài đặt ít nhất một GPU NVIDIA. Sau đó, tải xuống trình điều khiển NVIDIA và CUDA và làm theo lời nhắc để đặt đường dẫn thích hợp. Khi các bước chuẩn bị này hoàn tất, nvidia-smilệnh có thể được sử dụng để xem thông tin cạc đồ họa.

In [2]:
!nvidia-smi

Sun Mar  5 14:28:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Trong PyTorch, mọi mảng đều có một thiết bị, chúng tôi thường gọi nó là một ngữ cảnh. Cho đến nay, theo mặc định, tất cả các biến và tính toán liên quan đã được gán cho CPU. Thông thường, các bối cảnh khác có thể là các GPU khác nhau. Mọi thứ có thể trở nên rắc rối hơn khi chúng tôi triển khai công việc trên nhiều máy chủ. Bằng cách gán các mảng cho các ngữ cảnh một cách thông minh, chúng ta có thể giảm thiểu thời gian truyền dữ liệu giữa các thiết bị. Ví dụ: khi đào tạo mạng thần kinh trên máy chủ có GPU, chúng tôi thường muốn các tham số của mô hình tồn tại trên GPU.

Để chạy các chương trình trong phần này, bạn cần ít nhất hai GPU. Lưu ý rằng điều này có thể xa hoa đối với hầu hết các máy tính để bàn nhưng nó dễ dàng có sẵn trên đám mây, chẳng hạn như bằng cách sử dụng các phiên bản đa GPU AWS EC2. Hầu như tất cả các phần khác không yêu cầu nhiều GPU. Thay vào đó, điều này chỉ đơn giản là để minh họa cách dữ liệu di chuyển giữa các thiết bị khác nhau.

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

# 6.7.1. Thiết bị tính toán

Chúng tôi có thể chỉ định các thiết bị, chẳng hạn như CPU ​​và GPU, để lưu trữ và tính toán. Theo mặc định, các tenxơ được tạo trong bộ nhớ chính và sau đó sử dụng CPU để tính toán nó.

Trong PyTorch, CPU và GPU có thể được biểu thị bằng `torch.device('cpu')` và `torch.device('cuda')`. Cần lưu ý rằng cputhiết bị có nghĩa là tất cả các CPU và bộ nhớ vật lý. Điều này có nghĩa là các tính toán của PyTorch sẽ cố gắng sử dụng tất cả các lõi CPU. Tuy nhiên, một gputhiết bị chỉ đại diện cho một thẻ và bộ nhớ tương ứng. Nếu có nhiều GPU, chúng tôi sử dụng torch.device(f'cuda:{i}')để đại diện cho 
GPU $i^\mathrm{th}$ ($i$ bắt đầu từ 0). Ngoài ra, gpu:0 và gpu là tương đương.

In [5]:
def cpu():  
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0): 
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1), gpu(2)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1),
 device(type='cuda', index=2))

Chúng tôi có thể truy vấn số lượng GPU có sẵn.

In [7]:
def num_gpus(): 
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

1

Bây giờ chúng tôi xác định hai chức năng thuận tiện cho phép chúng tôi chạy mã ngay cả khi GPU được yêu cầu không tồn tại.

In [8]:
def try_gpu(i=0):  
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 6.7.2. Tensor và GPU

Theo mặc định, các tenxơ được tạo trên CPU. Chúng ta có thể truy vấn thiết bị nơi đặt tensor.


In [9]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

Điều quan trọng cần lưu ý là bất cứ khi nào chúng tôi muốn hoạt động trên nhiều thuật ngữ, chúng cần phải ở trên cùng một thiết bị. Chẳng hạn, nếu chúng ta tính tổng hai tenxơ, chúng ta cần đảm bảo rằng cả hai đối số đều nằm trên cùng một thiết bị—nếu không, khung sẽ không biết nơi lưu trữ kết quả hoặc thậm chí cách quyết định nơi thực hiện phép tính.

# 6.7.2.1. Lưu trữ trên GPU

Có một số cách để lưu trữ tensor trên GPU. Ví dụ: chúng ta có thể chỉ định thiết bị lưu trữ khi tạo tensor. Tiếp theo, chúng ta tạo biến tensor X trên biến đầu tiên gpu. Tenor được tạo trên GPU chỉ tiêu thụ bộ nhớ của GPU này. Chúng ta có thể sử dụng nvidia-smi lệnh để xem mức sử dụng bộ nhớ GPU. Nói chung, chúng tôi cần đảm bảo rằng chúng tôi không tạo dữ liệu vượt quá giới hạn bộ nhớ GPU.


In [10]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

Giả sử rằng bạn có ít nhất hai GPU, đoạn mã sau sẽ tạo một tensor ngẫu nhiên trên GPU thứ hai.

In [12]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y.device # do không có gpu thứ 2

device(type='cpu')

## 6.7.2.2. Sao chép

Nếu như ta mối tính toán ví dụ như cộng 2 tensor X + Y ở 2 GPU khác nhau, ta phải copy sang cùng một GPU rồi thực hiện phép tính.

![Copy data to perform an operation on the same device.](http://d2l.ai/_images/copyto.svg)


In [21]:
Z = Y.cuda()
print(Y)
print(Z)

tensor([[0.4622, 0.2333, 0.7128],
        [0.0933, 0.8761, 0.2962]])
tensor([[0.4622, 0.2333, 0.7128],
        [0.0933, 0.8761, 0.2962]], device='cuda:0')


In [22]:
X + Z

tensor([[1.4622, 1.2333, 1.7128],
        [1.0933, 1.8761, 1.2962]], device='cuda:0')